In [1]:
x <- read.csv("Intervals_122014.csv")

In [23]:
library(data.table,lib.loc="../Rpackages")
x_tmp <- fread("Intervals_122014.csv",header=TRUE,sep=",")
summary(x_tmp)
str(x_tmp)

Read 5124672 rows and 5 (of 5) columns from 0.282 GB file in 00:00:05


 MR_MDVC_NUMBER       START_READ          END_READ        INTERVAL_TIME     
 Length:5124672     Min.   :    0.45   Min.   :    0.45   Length:5124672    
 Class :character   1st Qu.: 5738.55   1st Qu.: 5754.67   Class :character  
 Mode  :character   Median : 8909.28   Median : 8929.35   Mode  :character  
                    Mean   : 9906.08   Mean   : 9929.12                     
                    3rd Qu.:12888.57   3rd Qu.:12914.62                     
                    Max.   :93837.32   Max.   :94035.96                     
                                                                            
 INTERVAL_READ    
 Min.   : 0.0000  
 1st Qu.: 0.0774  
 Median : 0.1536  
 Mean   : 0.2400  
 3rd Qu.: 0.3024  
 Max.   :13.8945  
 NA's   :960      

Classes 'data.table' and 'data.frame':	5124672 obs. of  5 variables:
 $ MR_MDVC_NUMBER: chr  "123053022" "123053022" "123053022" "123053022" ...
 $ START_READ    : num  10090 10090 10090 10090 10090 ...
 $ END_READ      : num  10116 10116 10116 10116 10116 ...
 $ INTERVAL_TIME : chr  "01-DEC-2014 00:15:00" "01-DEC-2014 00:30:00" "01-DEC-2014 00:45:00" "01-DEC-2014 01:00:00" ...
 $ INTERVAL_READ : num  0.1458 0.1128 0.093 0.0918 0.0912 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [24]:
length(unique(x$MR_MDVC_NUMBER))

[1] 1713

In [25]:
summary(x)
str(x)

 MR_MDVC_NUMBER        START_READ          END_READ       
 Min.   :123053022   Min.   :    0.45   Min.   :    0.45  
 1st Qu.:123053503   1st Qu.: 5738.55   1st Qu.: 5754.67  
 Median :123053944   Median : 8909.28   Median : 8929.35  
 Mean   :123056357   Mean   : 9906.08   Mean   : 9929.12  
 3rd Qu.:123054389   3rd Qu.:12888.57   3rd Qu.:12914.62  
 Max.   :125132704   Max.   :93837.32   Max.   :94035.96  
                                                          
              INTERVAL_TIME     INTERVAL_READ    
 01-DEC-2014 00:15:00:   1722   Min.   : 0.0000  
 01-DEC-2014 00:30:00:   1722   1st Qu.: 0.0774  
 01-DEC-2014 00:45:00:   1722   Median : 0.1536  
 01-DEC-2014 01:00:00:   1722   Mean   : 0.2400  
 01-DEC-2014 01:15:00:   1722   3rd Qu.: 0.3024  
 01-DEC-2014 01:30:00:   1722   Max.   :13.8945  
 (Other)             :5114340   NA's   :960      

'data.frame':	5124672 obs. of  5 variables:
 $ MR_MDVC_NUMBER: int  123053022 123053022 123053022 123053022 123053022 123053022 123053022 123053022 123053022 123053022 ...
 $ START_READ    : num  10090 10090 10090 10090 10090 ...
 $ END_READ      : num  10116 10116 10116 10116 10116 ...
 $ INTERVAL_TIME : Factor w/ 2976 levels "01-DEC-2014 00:15:00",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ INTERVAL_READ : num  0.1458 0.1128 0.093 0.0918 0.0912 ...


In [26]:
cvt <- function(date,fmt="%d-%b-%Y %H:%M:%S"){
    print(date)
    x <- strptime(date,fmt)
    print(x)
    unclass(as.POSIXct(x)+0);
}

In [27]:
x_tmp$INTERVAL_TIME[4]
(cvt(x_tmp$INTERVAL_TIME[4]))

[1] "01-DEC-2014 01:00:00"

[1] "01-DEC-2014 01:00:00"
[1] "2014-12-01 01:00:00 UTC"


[1] 1417395600

In [ ]:
x_tmp$INTERVAL_TIME <- sapply(x_tmp$INTERVAL_TIME, cvt)

In [ ]:
str(x_tmp)

In [ ]:
x <- x_tmp

In [11]:
#get 5 highest samples
tot = tapply(x$INTERVAL_READ, x$MR_MDVC_NUMBER, sum, na.rm=T)
qtot = quantile(tot,0:10/10);

o = order(-tot);
top = names(tot[o][1:5]);
x_top <- x[match(x$MR_MDVC_NUMBER, top, nomatch=0)>0,]

mid = names(tot[tot<qtot[6]&tot>=qtot[5]])[1:5];
x_mid <- x[match(x$MR_MDVC_NUMBER, mid, nomatch=0)>0,]

bot = names(tot[tot<qtot[3]&tot>=qtot[2]])[1:5];
x_bot <- x[match(x$MR_MDVC_NUMBER, bot, nomatch=0)>0,]

In [ ]:
my_plot <- function(tmp_df, i, q){
    name <- paste(i,"_",q,".jpeg",sep="")
    jpeg(name)
    par(mfrow=c(4,2))
    plot(filter(tmp_df[tmp_df$MR_MDVC_NUMBER==i,]$INTERVAL_READ,c(1,2,3,4,5,4,3,2,1)/(25)),main=i)
    acf(tmp_df[tmp_df$MR_MDVC_NUMBER==i,]$INTERVAL_READ,lag=24*4,main=i)
    acf(tmp_df[tmp_df$MR_MDVC_NUMBER==i,]$INTERVAL_READ,type="partial",lag=24*4,main=i)
    acf(diff(tmp_df$INTERVAL_READ),main=i)
    dev.off()
}

In [ ]:
unique(x_bot$MR_MDVC_NUMBER)
for(i in unique(x_bot$MR_MDVC_NUMBER)){
    my_plot(x_bot,i,"bot")
    print(i)
}
unique(x_top$MR_MDVC_NUMBER)
for(i in unique(x_top$MR_MDVC_NUMBER)){
    my_plot(x_top,i,"top")
    print(i)
}
unique(x_mid$MR_MDVC_NUMBER)
for(i in unique(x_mid$MR_MDVC_NUMBER)){
    my_plot(x_mid,i,"mid")
    print(i)
}

In [ ]:
my_acf <- function(tmp_df, i){
    r1<-acf(tmp_df[tmp_df$MR_MDVC_NUMBER==i,]$INTERVAL_READ,lag=24*4,plot=FALSE,na.action = na.contiguous)
    r2<-acf(tmp_df[tmp_df$MR_MDVC_NUMBER==i,]$INTERVAL_READ,type="partial",lag=24*4,plot=FALSE,na.action = na.contiguous)
    r3<-acf(diff(tmp_df$INTERVAL_READ),plot=FALSE,na.action = na.contiguous)
    return(list(r1,r2,r3))
}

In [ ]:
user <- data.frame(ID=integer(),total_power=integer(),acf_user=I(list()))
str(user)
colnames(user)

In [ ]:
counter <- 1
for(i in unique(x$MR_MDVC_NUMBER)){
    print(paste("ID: ",i))
    print(paste("total power: ", sum(x[x$MR_MDVC_NUMBER==i,]$INTERVAL_READ)))
    print(paste("length:",length(x[x$MR_MDVC_NUMBER==i,])))
    #print(counter)
    print(paste("acf list: ",my_acf(x,i)))
}
str(x)

In [ ]:
(t<-sum(x[x$MR_MDVC_NUMBER==123053233,]$INTERVAL_READ))
(l <- my_acf(x,123053233))
df <- data.frame()#ID=integer(123053233),total_power=double(t))
df$ID <- 123053233
df$total_power <- double(t)
df$acf_user <- l
str(df$ID)
str(df$total_power)
str(df$acf_user)

In [ ]:
df$ID
t